In [1]:
from glob import glob
from pathlib import Path

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow import keras
from tensorflow.keras import layers

## 画像から特徴量を取得

In [10]:
img_file_path_list = sorted(list(glob('../dataset/images/*.jpg')))
dst_size = [299, 299]
imgs = []
for i, file_path in enumerate(img_file_path_list):
    img = img_to_array(load_img(file_path, target_size=dst_size))
    imgs.append(img)

imgs = np.array(imgs, dtype=float)
imgs = preprocess_input(imgs)
img_feature_extractor = InceptionResNetV2(weights='imagenet', include_top=False)
img_features = img_feature_extractor.predict(imgs)

1/1 [==============================] - 2s 2s/step


## データセット生成

In [11]:
def load_html(path):
    with open(path, 'r') as f:
        text = f.read()
    return text

In [12]:
max_caption_len = 100
tokenizer = Tokenizer(filters='', split=' ', lower=False)
X = []
html_file_path_list = sorted(list(glob('../dataset/html/*.html')))
for html_file_path in html_file_path_list:
    X.append(load_html(html_file_path))
# 単語のインデックス化
tokenizer.fit_on_texts(X)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(X)
max_length = max(len(s) for s in sequences)

# モデルへの入力作成
X, y, image_data = [], [], []
# 1つのhtmlに1つの画像が対応
for img_idx, seq in enumerate(sequences):
    for i in range(1, len(seq)):
        in_seq, out_seq = seq[:i], seq[i]
        in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
        out_seq = keras.utils.to_categorical([out_seq], num_classes=vocab_size)[0]
        image_data.append(img_features[img_idx])
        X.append(in_seq[-max_caption_len:])
        y.append(out_seq)
X, y, image_data = np.array(X), np.array(y), np.array(image_data)
X.shape, y.shape, image_data.shape

((2306, 100), (2306, 436), (2306, 8, 8, 1536))

In [13]:
assert len(img_file_path_list) == len(html_file_path_list)
for img_path, html_path in zip(img_file_path_list, html_file_path_list):
    print(img_path, html_path)
    assert Path(img_path).stem == Path(html_path).stem

../dataset/images/86.jpg ../dataset/html/86.html
../dataset/images/87.jpg ../dataset/html/87.html
../dataset/images/88.jpg ../dataset/html/88.html
../dataset/images/89.jpg ../dataset/html/89.html
../dataset/images/90.jpg ../dataset/html/90.html


## モデル作成と学習

In [14]:
def build_model(
        max_caption_len=max_caption_len,
        vocab_size=vocab_size,
        embed_dim=200):
    # Encoder作成
    # 画像入力部分
    image_inputs = layers.Input(shape=(8, 8, 1536, ))
    image_flat = layers.Flatten()(image_inputs)
    image_flat = layers.Dense(128, activation='relu')(image_flat)
    image_outputs = layers.RepeatVector(max_caption_len)(image_flat)
    # テキスト入力部分
    text_inputs = layers.Input(shape=(max_caption_len, ))
    text_model = layers.Embedding(vocab_size, embed_dim, input_length=max_caption_len)(text_inputs)
    text_model = layers.LSTM(256, return_sequences=True)(text_model)
    text_model = layers.LSTM(256, return_sequences=True)(text_model)
    text_outputs = layers.Dense(128, activation='relu')(text_model)
    
    # Decoder作成
    decoder = layers.concatenate([image_outputs, text_outputs])
    decoder = layers.LSTM(512, return_sequences=False)(decoder)
    decoder_outputs = layers.Dense(vocab_size, activation='softmax')(decoder)

    # model complie
    model = keras.Model(inputs=[image_inputs, text_inputs], outputs=decoder_outputs)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model

In [15]:
checkpoint_dir_path = Path('checkpoint')
checkpoint_dir_path.mkdir(exist_ok=True)
model = build_model()
EPOCHS = 600
SAVE_PERIOD = 50
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath=str(checkpoint_dir_path / 'epoch_{epoch}'),
        save_freq='epoch',
        period=SAVE_PERIOD
    )
]
model.fit([image_data, X], y, batch_size=64, shuffle=False, epochs=EPOCHS, callbacks=callbacks)
model.save('final_model')

Epoch 1/600
37/37 [==============================] - 4s 35ms/step - loss: 5.9243
Epoch 2/600
37/37 [==============================] - 1s 35ms/step - loss: 5.7136
Epoch 3/600
37/37 [==============================] - 1s 35ms/step - loss: 5.7046
Epoch 4/600
37/37 [==============================] - 1s 35ms/step - loss: 5.6926
Epoch 5/600
37/37 [==============================] - 1s 35ms/step - loss: 5.6629
Epoch 6/600
37/37 [==============================] - 1s 35ms/step - loss: 5.6451
Epoch 7/600
37/37 [==============================] - 1s 35ms/step - loss: 5.6063
Epoch 8/600
37/37 [==============================] - 1s 37ms/step - loss: 5.6041
Epoch 9/600
37/37 [==============================] - 1s 35ms/step - loss: 5.5988
Epoch 10/600
37/37 [==============================] - 1s 35ms/step - loss: 5.5369
Epoch 11/600
37/37 [==============================] - 1s 35ms/step - loss: 5.5206
Epoch 12/600
37/37 [==============================] - 1s 36ms/step - loss: 5.5184
Epoch 13/600
37/37 [=====

INFO:tensorflow:Assets written to: checkpoint/epoch_50/assets


INFO:tensorflow:Assets written to: checkpoint/epoch_50/assets


37/37 [==============================] - 8s 213ms/step - loss: 5.0480
Epoch 51/600
37/37 [==============================] - 1s 35ms/step - loss: 5.0884
Epoch 52/600
37/37 [==============================] - 1s 37ms/step - loss: 5.0767
Epoch 53/600
37/37 [==============================] - 1s 36ms/step - loss: 5.0676
Epoch 54/600
37/37 [==============================] - 1s 37ms/step - loss: 5.0466
Epoch 55/600
37/37 [==============================] - 1s 36ms/step - loss: 5.0283
Epoch 56/600
37/37 [==============================] - 1s 36ms/step - loss: 5.0077
Epoch 57/600
37/37 [==============================] - 1s 35ms/step - loss: 5.0043
Epoch 58/600
37/37 [==============================] - 1s 39ms/step - loss: 4.9478
Epoch 59/600
37/37 [==============================] - 1s 35ms/step - loss: 4.9227
Epoch 60/600
37/37 [==============================] - 1s 35ms/step - loss: 4.9230
Epoch 61/600
37/37 [==============================] - 1s 39ms/step - loss: 4.8976
Epoch 62/600
37/37 [========

INFO:tensorflow:Assets written to: checkpoint/epoch_100/assets


...

INFO:tensorflow:Assets written to: checkpoint/epoch_600/assets


INFO:tensorflow:Assets written to: checkpoint/epoch_600/assets


37/37 [==============================] - 8s 223ms/step - loss: 8.5775e-04


INFO:tensorflow:Assets written to: final_model/assets


INFO:tensorflow:Assets written to: final_model/assets


## 学習済みモデルでHTML生成

In [16]:
def word_for_id(target_index, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == target_index:
            return word
    return None

In [35]:
def generate_desc(model, tokenizer, photo, max_length):
    # 初期トークン設定
    in_text = 'START'
    # 初期トークンを元に次の単語を予測していく
    for i in range(900):
        # トークンをindexに変換する。
        sequence = tokenizer.texts_to_sequences([in_text])[0][-max_length:]
        # max_lengthに満たない単語を0埋めする。
        sequence = pad_sequences([sequence], maxlen=max_length)
        # 次の単語予測
        yhat = model.predict([photo, sequence], verbose=0)
        # 予測確率 shape=(1, 語彙数)から最大のインデックスを取得
        yhat = np.argmax(yhat)
        # インデックスを単語に変換
        word = word_for_id(yhat, tokenizer)
        # 当てはめ対象がない場合は終了
        if word is None:
            break
        # 単語を追加
        in_text += ' ' + word
            
        # ストップワードが出力された場合は終了
        if word == 'END':
            break
        else:
            print(' ' + word, end='')
    return

In [36]:
model = tf.keras.models.load_model('./checkpoint/epoch_600')
test_image = img_to_array(load_img('../dataset/images/86.jpg', target_size=(299, 299)))
test_image = np.array(test_image, dtype=float)
test_image = preprocess_input(test_image)
test_features = img_feature_extractor.predict(np.array([test_image]), verbose=0)
generate_desc(model, tokenizer, np.array(test_features), 100)

 <!DOCTYPE html>
<html lang="en" dir="ltr">
<head>
<title>Basic 86</title>
<meta charset="iso-8859-1">
<link rel="stylesheet" href="styles/layout.css" type="text/css">
<!--[if lt IE 9]><script src="scripts/html5shiv.js"></script><![endif]-->
</head>
<body>
<div class="wrapper row1">
 <header id="header" class="clear">
 <div id="hgroup">
 <h1><a href="#">Basic 86</a></h1>
 <h2>Free HTML5 Website Template</h2>
 </div>
 <nav>
 <ul>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li class="last"><a href="#">Text Link</a></li>
 </ul>
 </nav>
 </header>
</div>
<!-- content -->
<div class="wrapper row2">
 <div id="container" class="clear">
 <!-- Slider -->
 <section id="slider" class="clear">
 <figure><img src="images/demo/630x300.gif" alt="">
 <figcaption>
 <h2>Eu justo augue estas</h2>
 <p>Nullamlacus dui ipsum conseque loborttis non euisque morbi penas dapibulum orna. Urnaultrices quis curabit